In [60]:
import os
os.chdir(r"C:\github_work\Conversational_AI\Source")

In [61]:
%pwd

'C:\\github_work\\Conversational_AI\\Source'

In [106]:
from config.settings_loader import load_config

config = load_config("config/config.yaml")

In [63]:
config["data_source"]["processed_data"]["petrol_vehicle_data"]

'data\\processed_data\\petrol_data.csv'

In [64]:
from exception.custom_exception import DocumentPortalException
from logger.custom_logger import logging

In [65]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [100]:
import pandas as pd
petrol_df = pd.read_csv(config["data_source"]["processed_data"]["petrol_vehicle_data"])
ev_df = pd.read_csv(config["data_source"]["processed_data"]["electric_vehicle_data"])
price_df = pd.read_csv(config["data_source"]["processed_data"]["price_data"])

In [107]:
# Map your DataFrames so the agent knows what they represent
prefix = """You are a professional customer care assistant with strong data analysis skills.

You have access to three pandas DataFrames that contain official product and pricing information.
You must answer customer questions strictly based on the data available in these DataFrames.

DataFrame mapping:
- df1: Petrol vehicle details (models, specifications, warranty, engine capacity, etc.)
- df2: Electric vehicle (EV) details (models, range, motor power, charging time, etc.)
- df3: Pricing information (showroom price, on-road price, taxes, and related costs)

Behavior guidelines:
- Act like a customer support representative assisting customers with product-related queries.
- Respond in a polite, clear, and customer-friendly manner.
- Use only the information present in the DataFrames; do not assume or fabricate details.
- If the requested information is not available in the data, clearly state that it is not available.
- Avoid technical jargon unless the customer explicitly asks for it.
- When appropriate, compare models to help customers make informed decisions.

Use the DataFrame names (df1, df2, df3) exactly as specified when reasoning about the data.
"""


agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model=config["model"]["name"]),
    df=[petrol_df, ev_df, price_df],
    agent_type="tool-calling",
    prefix=prefix,
    verbose=True,
    allow_dangerous_code=True
)

In [108]:
agent.invoke("How many total vehicles are listed in the petrol_data dataset?")



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'len(df1)'}`


9There are a total of 9 vehicles listed in the petrol_data dataset.

> Finished chain.


{'input': 'How many total vehicles are listed in the petrol_data dataset?',
 'output': 'There are a total of 9 vehicles listed in the petrol_data dataset.'}